In [ ]:
# !pip install -U adapter-transformers
# !conda install -y -c conda-forge tensorboard
# !conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch


In [ ]:
from datasets import load_dataset

scierc_name = 'nsusemiehl/SciERC'
scierc_dataset = load_dataset(scierc_name)
print(scierc_dataset.num_rows)

In [ ]:
scierc_dataset['train'][255]

In [ ]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

# Encode the input data
# NOTE: num_proc does not seem to work, for some reason it can't find the tokenizer
scierc_dataset = scierc_dataset.map(encode_batch, 
                                    batched=True, 
                                    remove_columns=scierc_dataset['train'].column_names, 
                                    )

# Main data processing function that will concatenate all texts from
# our dataset and generate chunks of block_size.

# NOTE: We may not need this due to our data being complete sentences
# block_size = 128
# def group_texts(examples):
#     concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
#     total_length = len(concatenated_examples[list(examples.keys())[0]])
#     result = {
#         k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
#         for k, t in concatenated_examples.items()
#     }
#     result["labels"] = result["input_ids"].copy()
#     return result

# NOTE: num_proc does not seem to work, for some reason it can't find the tokenizer  
# scierc_dataset = scierc_dataset.map(group_texts, batched=True)

def add_labels(examples):
  examples["labels"] = examples["input_ids"].copy()
  return examples
  
scierc_dataset = scierc_dataset.map(add_labels, batched=True)

In [ ]:
scierc_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


In [ ]:
# Collater adds padding in the form of EOS tokens, makes data augmentations of random masking ('mlm_probability)
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
# creating model
from transformers import RobertaConfig
from transformers import RobertaAdapterModel

config = RobertaConfig.from_pretrained(
    "roberta-base",
    # num_labels=num_of_labels,
)
model = RobertaAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

In [ ]:
# Add new adapter
model.add_adapter("sci_erc")
# Add a matching language model head
model.add_masked_lm_head(
    "sci_erc",
)
# Activate the adapter
# model.set_active_adapters("sci_erc")
model.train_adapter('sci_erc')


In [ ]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=10,
    output_dir="./training_output/pretraining",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,
    evaluation_strategy = 'steps',
    # load_best_model_at_end = True,
)


# Adding Support for Tensorboard, supposedly you don't have to do this, but I find that it doesn't work
from torch.utils.tensorboard import SummaryWriter
from transformers.integrations import TensorBoardCallback
writer = SummaryWriter()
writer = TensorBoardCallback(writer)

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=scierc_dataset["train"],
    eval_dataset=scierc_dataset["validation"],
    data_collator=data_collator,
     
)

In [9]:
trainer.train()

In [ ]:
trainer.evaluate(scierc_dataset['test'])